In [4]:
import json
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from functools import reduce

In [5]:
tickers = ["CASH",
    "AAPL", "MSFT", "NVDA", "ADBE", "CSCO",  # Tecnología
    "AMZN", "F", "MCD",                   # Consumo Discrecional
    "PG", "KO", "WMT",                       # Consumo Básico
    "JNJ", "PFE", "MRK",                     # Salud
    "JPM", "BAC", "GS",                      # Finanzas
    "XOM", "CVX", "COP",                     # Energía
    "GE", "MMM", "CAT",                      # Industriales
    "GOOGL", "IBM", "NFLX",                 # Comunicación
    "NEE", "AMT", "NUE",                      # Inmobiliario / Utilities / Materiales
]
ruta_m = "C:/Users/marco/Desktop/Cede/estudios/UNIR/TFM/Desarrollo/dashboards/data/raw/info_env0_test.json"

## Assets Weights

In [6]:
# 1) Carga el JSON (lista de dicts)
with open(ruta_m, 'r') as f:
    entradas = json.load(f)

# 2) Construye filas
filas = []
for e in entradas:
    # selecciona el bloque correcto
    info = e if 'date' in e else e.get('final_info', {})

    # extrae fecha
    fecha = info.get('date')
    assert fecha is not None, "No se encontró 'date' ni en final_info"

    # extrae pesos; puede venir ya como lista o como string
    raw = info.get('action', [])
    if isinstance(raw, str):
        # limpia corchetes y saltos, y parsea
        arr = np.fromstring(raw.strip().replace('\n', ' ').
                                  strip('[]'), sep=' ')
        pesos = arr.tolist()
    else:
        pesos = raw

    # validación
    assert len(pesos) == len(tickers), (
        f"¡Error! {len(pesos)} pesos pero {len(tickers)} tickers."
    )

    # arma la fila
    fila = {'date': fecha}
    for tk, w in zip(tickers, pesos):
        fila[tk] = w
    filas.append(fila)

# 3) DataFrame y CSV
df_alloc = pd.DataFrame(filas)
df_alloc.to_csv('../data/raw/asset_allocation.csv', index=False)

print(df_alloc.head())


         date      CASH      AAPL      MSFT      NVDA      ADBE      CSCO  \
0  2022-08-05  0.031528  0.031528  0.031528  0.031528  0.031528  0.085701   
1  2022-08-08  0.031528  0.031528  0.031528  0.031528  0.031528  0.031528   
2  2022-08-09  0.031528  0.031528  0.031528  0.031528  0.085701  0.031528   
3  2022-08-10  0.031528  0.031528  0.031528  0.031528  0.031528  0.031528   
4  2022-08-11  0.031528  0.031528  0.031528  0.031528  0.031528  0.031528   

       AMZN         F       MCD  ...       COP        GE       MMM       CAT  \
0  0.031528  0.031528  0.031528  ...  0.031528  0.031528  0.031528  0.031528   
1  0.031528  0.031528  0.031528  ...  0.031528  0.031528  0.031528  0.085701   
2  0.031528  0.031528  0.031528  ...  0.031528  0.031528  0.031528  0.031528   
3  0.031528  0.031528  0.031528  ...  0.031528  0.031528  0.031528  0.085701   
4  0.031528  0.031528  0.031528  ...  0.031528  0.031528  0.031528  0.085701   

      GOOGL       IBM      NFLX       NEE       AMT     

## Assets Prices

In [21]:
with open(ruta_m, 'r') as f:
    entradas = json.load(f)

filas = []

for i, e in enumerate(entradas):
    # Caso especial: el último elemento puede ser sólo 'final_info'
    if 'final_info' in e:
        info = e['final_info']
        fecha = info['date']
        raw_prices = info.get('post_c', [])
        # Parseo igual que abajo
        if isinstance(raw_prices, str):
            precios = np.fromstring(raw_prices.replace('\n', ' ').strip('[]'), sep=' ').tolist()
        else:
            precios = raw_prices
        assert len(precios) == len(tickers[1:]), f"¡Error! {len(precios)} precios pero {len(tickers[1:])} tickers[1:]."
        fila = {'date': fecha}
        for tk, p in zip(tickers[1:], precios):
            fila[tk] = p
        filas.append(fila)
        continue

    # Para el primer objeto, se añade dos filas:
    if i == 0:
        info = e.get('final_info', e)
        fecha_obj = info['date']

        # Fila con pre_c un día antes
        raw_pre = info.get('pre_c', [])
        # Convertir fecha a datetime y restar un día
        fecha_pre = (datetime.strptime(fecha_obj, "%Y-%m-%d") - timedelta(days=1)).strftime("%Y-%m-%d")
        if isinstance(raw_pre, str):
            precios = np.fromstring(raw_pre.replace('\n', ' ').strip('[]'), sep=' ').tolist()
        else:
            precios = raw_pre
        assert len(precios) == len(tickers[1:]), f"¡Error! {len(precios)} precios pero {len(tickers[1:])} tickers[1:]."
        fila = {'date': fecha_pre}
        for tk, p in zip(tickers[1:], precios):
            fila[tk] = p
        filas.append(fila)

        # Fila con post_c en la fecha normal
        raw_post = info.get('post_c', [])
        if isinstance(raw_post, str):
            precios = np.fromstring(raw_post.replace('\n', ' ').strip('[]'), sep=' ').tolist()
        else:
            precios = raw_post
        assert len(precios) == len(tickers[1:]), f"¡Error! {len(precios)} precios pero {len(tickers[1:])} tickers[1:]."
        fila = {'date': fecha_obj}
        for tk, p in zip(tickers[1:], precios):
            fila[tk] = p
        filas.append(fila)
        continue

    # Para los objetos normales
    info = e
    fecha = info['date']
    raw_prices = info.get('post_c', [])
    if isinstance(raw_prices, str):
        precios = np.fromstring(raw_prices.replace('\n', ' ').strip('[]'), sep=' ').tolist()
    else:
        precios = raw_prices
    assert len(precios) == len(tickers[1:]), f"¡Error! {len(precios)} precios pero {len(tickers[1:])} tickers[1:]."
    fila = {'date': fecha}
    for tk, p in zip(tickers[1:], precios):
        fila[tk] = p
    filas.append(fila)

# DataFrame y CSV
df_prices = pd.DataFrame(filas)
df_prices.to_csv('../data/raw/asset_prices.csv', index=False)
print(df_prices.head())

         date     AAPL     MSFT     NVDA    ADBE     CSCO    AMZN        F  \
0  2022-08-04  163.406  277.696  19.1904  430.59  44.9624  142.57  13.5877   
1  2022-08-05  163.179  276.974  18.9646  433.43  44.5958  140.80  13.5258   
2  2022-08-08  162.705  274.436  17.7703  434.34  44.6057  139.41  13.9503   
3  2022-08-09  162.755  276.375  17.0640  426.57  44.5066  137.83  13.4285   
4  2022-08-10  167.018  283.091  18.0739  438.40  45.5965  142.69  13.8394   

       MCD       PG  ...      COP       GE      MMM      CAT    GOOGL  \
0  257.769  143.631  ...  87.9254  58.4207  144.918  182.419  117.764   
1  256.375  143.700  ...  90.8308  58.9678  144.224  184.158  117.046   
2  253.972  144.246  ...  91.5939  59.6180  145.271  184.575  116.877   
3  257.195  143.909  ...  93.4329  59.4200  144.644  184.794  116.209   
4  258.481  145.269  ...  93.8928  61.1723  146.152  189.452  119.268   

       IBM    NFLX      NEE      AMT      NUE  
0  128.470  229.91  87.1130  269.306  133.62

## Portfolio History

In [7]:
# 1) Carga el JSON (lista de dicts)
with open(ruta_m, 'r') as f:
    entradas = json.load(f)

# 2) Construye filas para el historial
filas = []
for e in entradas:
    # Selecciona el bloque correcto (normal o final_info)
    info = e if 'date' in e else e.get('final_info', {})

    # Extrae fecha y valor cartera
    fecha = info.get('date')
    valor = info.get('portfolio_value')

    # Validación
    assert fecha is not None, "No se encontró 'date' ni en final_info"
    assert valor is not None, f"No se encontró 'portfolio_value' en la fecha {fecha}"

    filas.append({'date': fecha, 'portfolio_value': valor})

# 3) DataFrame y CSV
df_history = pd.DataFrame(filas)
df_history.to_csv('../data/raw/portfolio_history.csv', index=False)

print(df_history.head())


         date  portfolio_value
0  2022-08-05      1002.526766
1  2022-08-08      1002.380872
2  2022-08-09      1002.023654
3  2022-08-10      1020.854825
4  2022-08-11      1024.189278
